#Run this before starting
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"


In [71]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
import torch.optim as optim
from torch_geometric.nn.conv import feast_conv


In [72]:
import datetime
versj = 1
today = datetime.date.today().strftime("%b-%d")
print(today)

Jun-07


In [73]:
versj =2

In [74]:
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')

print("Using {}.".format(device_name))

Using cpu.


In [75]:
# Tester functions

# Test for new dataset

def testFinal(model, indata, device):
    model.eval()
    title = ''
    E_A = 0
    E_A_Ad = 0
    E_A_Ad2 = 0
    with torch.no_grad():
        for id, data in enumerate(indata):
          out = model(data.to(device))
          real = data.y.to(device)
          errorList = []
          error = 0
          epsilon = 1e-8
          for i in range(len(out)):
            e = ((out[i].item() - real[i].item() ) / (epsilon + out[i].item()))*100
            errorList.append(e)

          errorListAbs = [abs(x) for x in errorList]

          E_A_el = sum(errorListAbs)/len(errorListAbs)

          errorListAbsAd2 = errorListAbs[8:12]

          errorListAbs.sort()
          errorListAbsAd = errorListAbs[:-2]

          E_A_el = sum(errorListAbs)/len(errorListAbs)
          E_A_Ad_el = sum(errorListAbsAd)/len(errorListAbsAd)
          E_A_Ad_el2 = sum(errorListAbsAd2)/len(errorListAbsAd2)

          E_A += E_A_el
          E_A_Ad += E_A_Ad_el
          E_A_Ad2 += E_A_Ad_el2
          
          print([round(x,2) for x in errorList])
          print('Avarage Error: '+ str(round(E_A_el,2))+'%, Adjusted Avarage Error: '+ str(round(E_A_Ad_el,2))+'%')
          print('Mean Square: '+ str(round(F.mse_loss(out, data.y.to(device)).item(),4))+'\n')

          #errorAvg += sum(errorListAbs)/len(errorListAbs)

    print('Avarage Tot. Error: '+ str(round(E_A/len(indata),2))+'%\n')
    print('After removing the two worst errors. Avarage Adjusted Tot. Error: '+ str(round(E_A_Ad/len(indata),2))+'%')
    print('Total error for nodes 8, 9, 10, 11. Avarage Adjusted Tot. Error: '+ str(round(E_A_Ad2/len(indata),2))+'%')
   
#Test for new dataline

def testData(tester_model, datasetTest,line):
    real = datasetTest[line].y
    
    out = tester_model(datasetTest[line].to(device_name))
    Bool = ''
    print('{:<10}{:<10}{:<10}{:<10}'.format('real', 'out', '%error', 'Sign'))
    for i in range(len(out)):
        error = ((out[i].item() - real[i].item())/ out[i] )*100
        if torch.sign(real[i]) == torch.sign(out[i]):
            Bool = 'True'
        else:
            Bool = 'False'
        real_formatted = '{:<10.3f}'.format(round(real[i].item(),5))
        out_formatted = '{:<10.3f}'.format(round(out[i].item(),5))
        error_formatted = '{:<10.2f}'.format(error)
        list_formatted = [real_formatted, out_formatted, error_formatted, Bool]
        print('{:<10}{:<10}{:<10}{:<10}'.format(*list_formatted))

In [76]:
# Get data sets

import numpy as np
import torch
import random
from torch_geometric.data import Data, DataLoader

#Import data function
def get_data(filepath):
    data = np.loadtxt(filepath)
    x = data[:, :20]
    z = data[:, 20:40]
    m = data[:, 40:60]

    dataX = []
    for i in range(x.shape[0]):
        dataXs = np.column_stack((np.arange(20), x[i], z[i]))
        dataX.append(dataXs)

    dataY = m
    dataX = np.array(dataX)
    dataY = np.array(dataY)

    dataEdgeIndex = np.column_stack((np.arange(20)[:-1], np.arange(20)[1:]))
    dataEdgeIndex = np.vstack((dataEdgeIndex, dataEdgeIndex[:, ::-1])).T

    dataset = [Data(x=torch.from_numpy(x).float(), edge_index=torch.from_numpy(dataEdgeIndex).long(), y=torch.from_numpy(y).float()) for x, y in zip(dataX, dataY)]

    return dataset

#Getting data
datasetTrain = get_data("Data\\1DatasetTrain.txt")
datasetTest = get_data("Data\\1DatasetTest.txt")
datasetNew = get_data("Data\\1DatasetNew.txt")

In [77]:
data_outside = get_data("Data\\dataoutside.txt")
data_inside = get_data("Data\\datainside.txt")

In [78]:
# Data for test arch

comparison_data = np.loadtxt("Data\\comparison_test.txt")
compar_x = comparison_data[:20]
compar_z = comparison_data[20:40]
compar_m = comparison_data[40:60]

comparison_x = np.zeros((20, 3))
comparison_y = np.zeros((20))


for i in range(0,20):
    comparison_x[i][0] = i
    comparison_x[i][1] = compar_x[i]
    comparison_x[i][2] = compar_z[i]

    comparison_y[i] = compar_m[i]

comparison_arch = Data(x=torch.from_numpy(comparison_x).float(), edge_index=data_outside[0].edge_index,y=torch.from_numpy(comparison_y).float())
print(comparison_arch.x)


tensor([[  0.0000,   0.0000,   0.0000],
        [  1.0000,   2.8740,   6.5474],
        [  2.0000,   6.5638,  12.6722],
        [  3.0000,  11.0085,  18.2734],
        [  4.0000,  16.1346,  23.2585],
        [  5.0000,  21.8577,  27.5451],
        [  6.0000,  28.0831,  31.0625],
        [  7.0000,  34.7081,  33.7527],
        [  8.0000,  41.6234,  35.5712],
        [  9.0000,  48.7148,  36.4880],
        [ 10.0000,  55.8652,  36.4880],
        [ 11.0000,  62.9566,  35.5712],
        [ 12.0000,  69.8719,  33.7527],
        [ 13.0000,  76.4969,  31.0625],
        [ 14.0000,  82.7223,  27.5451],
        [ 15.0000,  88.4454,  23.2585],
        [ 16.0000,  93.5715,  18.2734],
        [ 17.0000,  98.0162,  12.6722],
        [ 18.0000, 101.7060,   6.5474],
        [ 19.0000, 104.5800,   0.0000]])


In [79]:
print(len(datasetNew))
print(len(datasetTest))
print(len(datasetTrain))

#Splitting dataset, 75% for training and 25% for validation

train_loader = datasetTrain[:int(0.75*len(datasetTrain))]
test_loader = datasetTrain[int(0.75*len(datasetTrain)):]

train_loader = datasetTrain[:1000]
test_loader = datasetTrain[-250:]

print(len(train_loader))
print(len(test_loader))

print(datasetTrain[10].edge_index.shape)
print(datasetTrain[10].x.shape)
print(datasetTrain[10].y.shape)

100
1001
6571
1000
250
torch.Size([2, 38])
torch.Size([20, 3])
torch.Size([20])


In [93]:
# ArchNN model architecture

# Importing necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn.conv import feast_conv 

class ArchNN(torch.nn.Module):
    def __init__(self, in_channels, heads, t_inv = True):
        super(ArchNN, self).__init__()
        
        # This fully connected layer transforms the input to have 16 features
        self.fc0 = nn.Linear(in_channels, 16)
        
        # FeastConv convolutional layers
        # Each layer doubles the number of features until reaching 256
        self.conv1 = feast_conv.FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = feast_conv.FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = feast_conv.FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        self.conv4 = feast_conv.FeaStConv(128, 256, heads=heads, t_inv=t_inv)
        
        # This layer halves the number of features to 128
        self.conv5 = feast_conv.FeaStConv(256, 128, heads=heads, t_inv=t_inv)
        
        # Fully connected layers reduce the dimensionality of the output
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)

        # Resets the parameters of the convolutional layers
        self.reset_parameters()

    # The reset_parameters function
    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()
        self.conv4.reset_parameters()
        self.conv5.reset_parameters()

    # The forward function passes the data through the neural network
    def forward(self, data):
        # Getting the node_feature tensor and edge_index tensor
        x, edge_index = data.x, data.edge_index
        
        # node_feature tensor is passed through the initial fully connected layer
        x = F.leaky_relu(self.fc0(x))
        
        # node_feature and edge_index tensors are passed through the 5 convolutional layers
        x = F.leaky_relu(self.conv1(x, edge_index))
        x = F.leaky_relu(self.conv2(x, edge_index))
        x = F.leaky_relu(self.conv3(x, edge_index))
        x = F.leaky_relu(self.conv4(x, edge_index))
        x = F.leaky_relu(self.conv5(x, edge_index))
        
        # The output of the convolutions is passed through two fully connected layers
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        
        # This step removes singleton dimensions
        x = torch.squeeze(x, dim=1)
        
        # The output of the network is returned
        return x

# 6 layers model

class ArchNN3(torch.nn.Module):
    def __init__(self, in_channels, heads, t_inv = True):
        super(ArchNN3, self).__init__()
        self.fc0 = nn.Linear(in_channels, 16)
        self.conv1 = feast_conv.FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = feast_conv.FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = feast_conv.FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        self.conv4 = feast_conv.FeaStConv(128, 256, heads=heads, t_inv=t_inv)
        self.conv5 = feast_conv.FeaStConv(256, 512, heads=heads, t_inv=t_inv)
        self.conv6 = feast_conv.FeaStConv(512, 128, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)

        self.reset_parameters()

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()
        self.conv4.reset_parameters()
        self.conv5.reset_parameters()
        self.conv6.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.leaky_relu(self.fc0(x))
        x = F.leaky_relu(self.conv1(x, edge_index))
        x = F.leaky_relu(self.conv2(x, edge_index))
        x = F.leaky_relu(self.conv3(x, edge_index))
        x = F.leaky_relu(self.conv4(x, edge_index))
        x = F.leaky_relu(self.conv5(x, edge_index))
        x = F.leaky_relu(self.conv6(x, edge_index))
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.squeeze(x, dim=1)
        return x

# ReLU model
class ArchNN2(torch.nn.Module):
    def __init__(self, in_channels, heads, t_inv = True):
        super(ArchNN2, self).__init__()
        self.fc0 = nn.Linear(in_channels, 16)
        self.conv1 = feast_conv.FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = feast_conv.FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = feast_conv.FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        self.conv4 = feast_conv.FeaStConv(128, 256, heads=heads, t_inv=t_inv)
        self.conv5 = feast_conv.FeaStConv(256, 128, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)

        self.reset_parameters()

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()
        self.conv4.reset_parameters()
        self.conv5.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.fc0(x))
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        x = F.relu(self.conv4(x, edge_index))
        x = F.relu(self.conv5(x, edge_index))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.squeeze(x, dim=1)
        return x
    

In [87]:
# Importing necessary libraries
import time
import torch
import torch.nn.functional as F

# This function prints the current state of training
def print_info(info):
    message = ('Epoch: {}/{}, Duration: {:.3f}s,'
               'Train Loss: {:.4f}, Test Loss:{:.4f}, Test Percentage Loss: {:.2f}%').format(
                   info['current_epoch'], info['epochs'], info['t_duration'],
                   info['train_loss'], info['test_loss'], info['test_percentage_loss'])
    print(message)

# This is the main function to run the training and testing of the model
def run(model, train_loader, test_loader, epochs, optimizer, device, lossfunc):
    # Initialize lists to hold loss values for each epoch
    train_losses = []
    test_losses = []
    test_percentage_losses = []

    # Loop over each epoch
    for epoch in range(1, epochs + 1):
        t = time.time()

        # Train the model on the training data
        train_loss = train(model, train_loader, optimizer, device, lossfunc)

        # Calculate the duration of the training
        t_duration = time.time() - t

        # Test the model on the test data
        test_loss, test_percentage_loss = test(model, test_loader, device, lossfunc)

        # Gather information about the current epoch
        eval_info = {
            'train_loss': train_loss,
            'test_loss': test_loss,
            'test_percentage_loss': test_percentage_loss,
            'current_epoch': epoch,
            'epochs': epochs,
            't_duration': t_duration
        }

        # Print the current state of training
        print_info(eval_info)

        # Append the losses for this epoch to the lists
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        test_percentage_losses.append(test_percentage_loss)

        if test_percentage_loss < 3:
            return train_losses, test_losses, test_percentage_losses

    # Return the lists of losses after training is finished
    return train_losses, test_losses, test_percentage_losses


# This function trains the model for one epoch
def train(model, train_loader, optimizer, device, lossfunc):
    model.train()

    total_loss = 0

    # Loop over each batch in the training data
    for idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))

        # Compute the mean squared error loss
        losses = lossfunc(output, data.y.to(device), reduction='none')  

        # Compute mean loss for backpropagation
        loss = losses.mean()

        # Backpropagate the gradients
        loss.backward(retain_graph=True)

        # Update the model parameters
        optimizer.step()

        # Add the loss for this batch to the total loss
        total_loss += loss.item()
    
    # Return the average loss for this epoch
    return total_loss / len(train_loader)

# This function tests the model
def test(model, test_loader, device, lossfunc):
    model.eval()

    total_loss = 0

    total_percentage_loss = 0

    # Ensure no gradients are calculated
    with torch.no_grad():

        # Loop over each batch in the test data
        for idx, data in enumerate(test_loader):
            out = model(data.to(device))
            real = data.y.to(device)

            # Calculate the loss for this batch and add it to the total loss
            total_loss += lossfunc(out, real).item()

            errorList = []

            epsilon = 1e-8
            for i in range(len(out)):
                e = ((out[i].item() - real[i].item() ) / (epsilon + out[i].item()))*100
                errorList.append(e)

            errorList = [abs(x) for x in errorList]

            total_percentage_loss += sum(errorList)/len(errorList)
    
    # Return the average loss for the test data
    return total_loss / len(test_loader), total_percentage_loss / len(test_loader)

In [ ]:
#runner
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

model = ArchNN(num_features, heads=8).to(device_name)

optimizer = optim.Adam(model.parameters(),
                       lr=0.0001)

lossfunc = F.mse_loss

train_losses, test_losses, test_percentage_losses = \
run(model, train_loader, test_loader, 1000, optimizer, device_name, lossfunc)

# versj defined at the top of the notebook
modelPath = "trained_models\\" + today + "ArchGNN" + str(versj) +".pt"
trainLossPath = "trained_models\\" + today + "ArchGNN" + str(versj) + "TrainLoss.txt"
testLossPath = "trained_models\\" + today + "ArchGNN" + str(versj) + "TestLoss.txt"
testPercLossPath = "trained_models\\" + today + "ArchGNN" + str(versj) + "TestPercentageLoss.txt"

np.savetxt(trainLossPath, np.array(train_losses))
np.savetxt(testLossPath, np.array(test_losses))
np.savetxt(testPercLossPath, np.array(test_percentage_losses))

torch.save(model.state_dict(), modelPath)
versj += 1
print("saved as: ", modelPath)

In [89]:
#load model
modelRelu_dict = torch.load("trained_models\\May-19ArchGNNReLUVersion2.pt")
modelLeaky_dict = torch.load("trained_models\\May-19ArchGNNVersion1.pt")

In [97]:
num_features = train_loader[0].x.shape[1]

modelRelu = ArchNN2(num_features, heads=8).to(device_name)
modelRelu.load_state_dict(modelRelu_dict)
modelRelu.eval()

ArchNN2(
  (fc0): Linear(in_features=3, out_features=16, bias=True)
  (conv1): FeaStConv(16, 32, heads=8)
  (conv2): FeaStConv(32, 64, heads=8)
  (conv3): FeaStConv(64, 128, heads=8)
  (conv4): FeaStConv(128, 256, heads=8)
  (conv5): FeaStConv(256, 128, heads=8)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [95]:
modelLeaky = ArchNN(num_features, heads=8).to(device_name)
modelLeaky.load_state_dict(modelLeaky_dict)
modelLeaky.eval()

ArchNN(
  (fc0): Linear(in_features=3, out_features=16, bias=True)
  (conv1): FeaStConv(16, 32, heads=8)
  (conv2): FeaStConv(32, 64, heads=8)
  (conv3): FeaStConv(64, 128, heads=8)
  (conv4): FeaStConv(128, 256, heads=8)
  (conv5): FeaStConv(256, 128, heads=8)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [98]:
num_features = train_loader[0].x.shape[1]

modelBest_dict = torch.load("trained_models\\May-26ArchGNNLeaky3.pt")

modelBest = ArchNN(num_features, heads=8).to(device_name)
modelBest.load_state_dict(modelBest_dict)
modelBest.eval()

ArchNN(
  (fc0): Linear(in_features=3, out_features=16, bias=True)
  (conv1): FeaStConv(16, 32, heads=8)
  (conv2): FeaStConv(32, 64, heads=8)
  (conv3): FeaStConv(64, 128, heads=8)
  (conv4): FeaStConv(128, 256, heads=8)
  (conv5): FeaStConv(256, 128, heads=8)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [99]:
for i in range(0, 1001):
    #print(i)
    #print(datasetTest[i].x[19][1].item())
    #print(datasetTest[i].x[int(9)][2].item())
    if (datasetTest[i].x[int(9)][2].item() < -85):
        print(i)

725


In [100]:
archnr = 725

print(datasetTest[archnr].x[19][1].item())
print(datasetTest[archnr].x[int(9)][2].item())
moments = np.array(datasetTest[archnr].y.tolist())
preds = np.array(modelBest(datasetTest[archnr]).tolist())
print(moments)
print(preds)
saver = np.concatenate((moments, preds))
saver = np.round(saver, 3)
print(saver)




175.0
-85.4587173461914
[ 7.32631302 -1.63355803  1.96350205  3.6179359   3.65740108  2.53618193
  0.77502501 -1.10456097 -2.64430189 -3.50446892 -3.50446892 -2.64430189
 -1.10456097  0.77502501  2.53618193  3.65740108  3.6179359   1.96350205
 -1.63355899 -7.32631302]
[ 7.30266762 -1.62797511  1.95984697  3.61372566  3.6466608   2.51864076
  0.76446778 -1.09951007 -2.62072182 -3.48917198 -3.48914838 -2.62682915
 -1.09899735  0.77032971  2.52975464  3.63680506  3.59970903  1.96599352
 -1.6284579  -7.28978777]
[ 7.326 -1.634  1.964  3.618  3.657  2.536  0.775 -1.105 -2.644 -3.504
 -3.504 -2.644 -1.105  0.775  2.536  3.657  3.618  1.964 -1.634 -7.326
  7.303 -1.628  1.96   3.614  3.647  2.519  0.764 -1.1   -2.621 -3.489
 -3.489 -2.627 -1.099  0.77   2.53   3.637  3.6    1.966 -1.628 -7.29 ]


In [103]:
# save data for arch labels and predictions for structural response animation in Grasshopper
# file: Data\\length_height.txt

np.savetxt("Data\\175_-85.txt", saver)

In [104]:
comparison_dataset = [comparison_arch]
prediction = modelBest(comparison_arch)
print(prediction)
print(comparison_arch.y)


tensor([ 0.8339,  0.1383,  0.6443,  0.7832,  0.6608,  0.3452, -0.0590, -0.4381,
        -0.7376, -0.8946, -0.9087, -0.7365, -0.4342, -0.0505,  0.3501,  0.6581,
         0.7893,  0.6430,  0.1216, -0.8530], grad_fn=<SqueezeBackward1>)
tensor([ 0.8436,  0.1256,  0.6442,  0.7901,  0.6560,  0.3420, -0.0529, -0.4385,
        -0.7401, -0.9047, -0.9047, -0.7401, -0.4385, -0.0529,  0.3420,  0.6560,
         0.7901,  0.6442,  0.1256, -0.8436])


In [105]:
print(len(data_outside))
testFinal(modelBest, data_outside, device_name)


286
[-3.47, -4.49, -3.18, -3.41, -1.96, -1.21, -7.45, -0.99, -14.07, -11.65, -8.59, -4.12, -5.89, -0.85, -8.1, -7.2, -7.04, -5.02, 20.03, -6.91]
Avarage Error: 6.28%, Adjusted Avarage Error: 5.08%
Mean Square: 0.0915

[-0.46, -0.63, -0.5, -0.24, -0.27, -0.61, -2.03, -0.57, -1.11, -1.09, -0.76, -0.92, -0.58, -0.52, -0.4, -0.82, -1.12, 0.28, -0.95, -0.79]
Avarage Error: 0.73%, Adjusted Avarage Error: 0.64%
Mean Square: 0.0006

[-0.71, -0.51, -0.02, -0.42, -0.36, -0.24, -0.76, -0.23, -0.33, 0.0, -0.08, -0.13, -0.27, 0.92, 0.29, -0.02, -0.02, 0.33, -0.82, -0.06]
Avarage Error: 0.33%, Adjusted Avarage Error: 0.27%
Mean Square: 0.0001

[-1.72, 38.95, -0.56, -0.86, -0.54, -0.99, 8.32, -0.22, -0.81, -0.13, -0.11, 0.4, 0.17, -208.0, 0.75, 0.73, -0.02, 0.3, 20.23, -0.62]
Avarage Error: 14.22%, Adjusted Avarage Error: 2.08%
Mean Square: 0.0001

[2.72, 0.68, -0.22, -0.33, -1.39, -5.19, -1.65, -0.04, -1.02, -0.79, 0.64, -0.1, 0.28, 0.05, -9.61, -0.32, -0.16, 0.85, -0.23, 7.87]
Avarage Error: 1.71%,

In [106]:
testFinal(modelBest, datasetTest, device_name)

[0.96, 1.17, 0.6, 1.23, 17.22, 2.23, 0.32, 0.91, 0.7, 0.85, 0.91, 0.8, 0.4, 0.07, 1.1, -6.14, -1.5, -1.46, -1.98, -1.36]
Avarage Error: 2.1%, Adjusted Avarage Error: 1.03%
Mean Square: 0.001

[-0.51, -1.33, -0.3, -0.16, -0.19, -0.1, 0.09, -0.5, -0.45, -0.42, -0.39, -0.31, -0.27, -0.02, 0.33, -0.22, -0.32, 0.28, -0.45, 0.04]
Avarage Error: 0.33%, Adjusted Avarage Error: 0.27%
Mean Square: 0.0001

[-2.66, -0.81, 0.03, -0.09, -2.03, -6.86, -1.18, -1.86, -1.21, -1.27, -1.79, -0.93, -2.98, -1.98, -10.3, 1.04, -1.94, -1.56, -0.02, 0.35]
Avarage Error: 2.04%, Adjusted Avarage Error: 1.32%
Mean Square: 0.0001

[-8.56, 1.03, -0.1, 0.58, 1.82, 16.28, -1.77, -1.68, -1.87, -0.92, -1.18, -0.53, -0.49, -1.51, 4.74, 0.58, -0.48, -0.74, -1.9, 2.38]
Avarage Error: 2.46%, Adjusted Avarage Error: 1.35%
Mean Square: 0.0001

[-1.64, -0.39, -0.16, -0.51, 0.43, -4.75, -1.73, -1.29, -0.21, 0.55, 1.22, 0.29, 0.1, 0.16, -0.17, 0.18, -0.47, -0.58, -0.62, -0.36]
Avarage Error: 0.79%, Adjusted Avarage Error: 0.52%

In [109]:
# printing parameters

def count_parameters(model):
    """Count the number of trainable parameters in a PyTorch model."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


# Print the number of parameters
print("Number of parameters in the model: {}".format(count_parameters(modelBest)))

Number of parameters in the model: 623305
